In [ ]:
import pandas as pd
from census import Census
# Get Variable Code
def get_acs_code(var_select,filepath = '../data/acs_variable_code.csv'):
    acs_code_df = pd.read_csv(filepath)
    acs_dict = {}
    for i in range(len(acs_code_df)):
        acs_dict[acs_code_df.loc[i,'Variable Name']] = acs_code_df.loc[i,'Variable Code']

    code_list = ['NAME']
    for f in var_select:
        code_list.append(acs_dict[f])
    return tuple(code_list), acs_dict

code_list,acs_dict = get_acs_code(var_select=['Total Population', 'Total Housing Units'],filepath = '../data/acs_variable_code.csv')
print(code_list)
import requests
def get_fips_code(lat, lon):
    url = "https://geo.fcc.gov/api/census/block/find"
    params = {
        "latitude": lat,
        "longitude": lon,
        "format": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    state_fips = data["State"]["FIPS"]
    state_name = data["State"]["name"]
    county_fips = data["County"]["FIPS"]
    county_name = data["County"]["name"]
    return state_fips,state_name, county_fips, county_name

def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

api_file = '/Users/caoguangyue/GitHub/Urban-Design-Equity-Metrics/app/census_api_key.txt'
api_key = get_file_contents(api_file)
c = Census(api_key)
# Replace with Input from lat, lng
state_fips, state_name, county_fips, county_name = get_fips_code(37.7749, -122.4194)
print(code_list)
print(state_fips, county_fips)


sc_census = c.acs5.state_county_blockgroup(fields= code_list, 
                                           state_fips = str(state_fips), 
                                           county_fips = str(county_fips[2::]), 
                                           blockgroup = '*', 
                                           year = 2019)

#### 2. Feature engineering
# Feature Engineering
acs_df = pd.DataFrame(sc_census)
acs_df['geoid'] = acs_df['state'] + acs_df['county'] + acs_df['tract'] + acs_df['block group']


inv_acs_dict = {v: k for k, v in acs_dict.items()}
acs_df.rename(columns=inv_acs_dict, inplace=True)

acs_df.head()